In [1]:
import pandas as pd
import numpy as np
from m6_feats_comb import *
from m7_utils import *

In [2]:
INPUT_DIR = 'kaggle/input/linking-writing-processes-to-writing-quality'
FEAT_STORE_DIR = 'feat_store_combined'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

logs = pd.concat([train_logs, test_logs], axis=0)
logs = normalise_up_down_times(logs)

In [31]:
train_ids = train_logs.id
test_ids = test_logs.id

from m3_model_params import lgb_params_1 as params

base_feats = pd.read_pickle(f'{FEAT_STORE_DIR}/train_base_feats.pkl')

compare_with_baseline_comb(base_dir, base_feats, train_ids, test_ids, params, 0.6100, train_scores)

NameError: name 'compare_with_baseline_comb' is not defined

In [3]:
train_ids = train_logs.id
test_ids = test_logs.id

logs = pd.concat([train_logs, test_logs], axis=0)
logs = normalise_up_down_times(logs)

preprocessor = Preprocessor(seed=42)
feats = preprocessor.make_feats(logs)
nan_cols = feats.columns[feats[feats['id'].isin(train_ids)].isna().any()].tolist()
feats = feats.drop(columns=nan_cols)

essays = getEssays(logs)
sent_feats = compute_sentence_aggregations(essays)
par_feats = compute_paragraph_aggregations(essays)
word_feats = create_word_length_features(essays, 'essay', 'id', 'essay_words')
vector_feats = countvectorize_one_one(logs)
at_by_activity = process_action_time_activity(logs)

feats = feats.merge(sent_feats, on='id', how='left')
feats = feats.merge(par_feats, on='id', how='left')
feats = feats.merge(word_feats, on='id', how='left')
feats = pd.concat([feats, vector_feats], axis=1) # tok 17 forward should disspear
feats = feats.merge(at_by_activity, on='id', how='left')

feats.to_pickle('scaled_feats.pkl')

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 33/33 [01:26<00:00,  2.61s/it, column=word_count_change100, method=kurt]         


Engineering activity counts data


100%|██████████| 2474/2474 [00:00<00:00, 13808.03it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering event counts data


...

100%|██████████| 2474/2474 [00:00<00:00, 13864.60it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering punctuation counts data


100%|██████████| 2474/2474 [00:00<00:00, 12795.44it/s]


Engineering input words data
Engineering ratios data


/root/Projects/Kaggle/linking-writing/m6_feats_comb.py:418: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
/root/Projects/Kaggle/linking-writing/m6_feats_comb.py:419: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
/root/Projects/Kaggle/linking-writing/m6_feats_comb.py:420: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

In [4]:
def agg_fe_df(logs):
    agg_fe_df = logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(
        ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum']) # what about quantiles, IQR, etc.
    
    agg_fe_df.columns = ['_'.join(x) for x in agg_fe_df.columns]
    agg_fe_df = agg_fe_df.add_prefix("tmp_")
    agg_fe_df.reset_index(inplace=True)
    return agg_fe_df

agg_fe_df(logs)

,id,tmp_down_time_mean,tmp_down_time_std,tmp_down_time_min,tmp_down_time_max,tmp_down_time_last,tmp_down_time_first,tmp_down_time_sem,tmp_down_time_median,tmp_down_time_sum,...,tmp_cursor_position_sum,tmp_word_count_mean,tmp_word_count_std,tmp_word_count_min,tmp_word_count_max,tmp_word_count_last,tmp_word_count_first,tmp_word_count_sem,tmp_word_count_median,tmp_word_count_sum
0,0000aaaa,2.108200e+05,298144.503219,0,421640,421640,0,210820.000000,210820.0,421640,...,1,0.000000,0.000000,0,0,0,0,0.000000,0.0,0
1,001519c8,8.436548e+05,395112.665961,0,1797351,1797351,0,7813.679400,887190.0,2157225252,...,1818445,128.116152,76.498372,0,256,255,0,1.512819,132.0,327593
2,0022f953,4.882323e+05,384959.404177,0,1758219,1758219,0,7771.013336,377050.0,1198122181,...,1904809,182.714751,97.763090,0,323,320,0,1.973502,186.0,448382
3,0042269b,8.240508e+05,489500.796565,0,1766778,1766778,0,7611.375322,755141.0,3408274006,...,3025946,194.772727,108.935068,0,404,404,0,1.693860,193.0,805580
4,0059420b,7.440880e+05,385205.014399,0,1362999,1362999,0,9765.334758,806845.5,1157800969,...,844188,103.618895,61.882250,0,206,206,0,1.568777,108.5,161231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2469,ffb8c745,7.136349e+05,503882.020411,0,1769114,1769114,0,7319.568976,712628.0,3381915633,...,3667989,256.353661,118.093794,0,461,273,0,1.715472,297.0,1214860
2470,ffbef7e5,8.202342e+05,512744.745940,0,1777392,1777392,0,10048.025509,726672.5,2135889912,...,2661493,223.013057,126.627934,0,438,438,0,2.481470,227.5,580726
2471,ffccd6fd,1.205533e+06,514320.848199,0,1935791,1935791,0,9293.100430,1483043.0,3692547571,...,4009729,157.589292,61.236111,0,201,201,0,1.106456,201.0,482696
2472,ffec5b38,5.566335e+05,334477.976640,0,1488450,1488450,0,5874.366278,554027.0,1804605942,...,3866542,205.917027,118.473905,0,413,413,0,2.080732,205.0,667583


In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()

feats = pd.read_pickle('scaled_feats.pkl')
at_activity = process_action_time_activity(logs)
feats = feats.merge(at_activity, on='id', how='left')

agg_fe_df = agg_fe_df(logs)
feats = feats.merge(agg_fe_df, on='id', how='left')

feats.replace([np.inf, -np.inf], np.nan, inplace=True)
feats.fillna(-1e10,inplace=True)
feats_to_transform = feats.drop(columns='id')
feats_columns = feats_to_transform.columns
feats_to_transform = scaler.fit_transform(feats_to_transform)
feats_transf = pd.DataFrame(data=feats_to_transform, columns=feats_columns)
feats = pd.concat([feats['id'], feats_transf], axis=1)

In [6]:
from m5_models import *
from m3_model_params import lgb_params_2 as lgbm_params
from m3_model_params import xgb_params_2 as xgb_params

train_ids = train_logs.id
test_ids = test_logs.id

feats = feats.merge(train_scores, on='id', how='left')
feats['low_sent_count'] = feats['low_sent_count'].astype(bool)

train_feats = feats[feats['id'].isin(train_ids)]
test_feats = feats[feats['id'].isin(test_ids)]

_, oof_1, rmse, model1 = xgb_cv_pipeline(train_feats=train_feats, 
                                        test_feats=test_feats, 
                                        xgb_params=xgb_params, 
                                        seed=42, 
                                        n_repeats=5, 
                                        n_splits=6)
                                        
_, oof_2, rmse, model1 = cv_pipeline(train_feats, test_feats, lgbm_params, lgbm_params['boosting_type'])

blend = pd.concat([oof_1, oof_2], axis=0)
blend_scores = blend.groupby(['id','score'])['prediction'].mean().reset_index()
blend_rmse = mean_squared_error(blend_scores['score'], blend_scores['prediction'], squared=False)
print(f'Blend RMSE {blend_rmse:.6f}')

/root/Projects/Kaggle/linking-writing/m5_models.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_feats.replace([np.inf, -np.inf], np.nan, inplace=True)
Iterations:   0%|          | 0/5 [00:00<?, ?it/s]/root/miniconda3/envs/lrp/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:38:50] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
Iterations: 100%|██████████| 5

XGB Average RMSE over 30 folds: 0.614286
LGBM Average RMSE over 50 folds: 0.613807
Blend RMSE 0.610875


In [ ]:
# 0.608132 - StandardScaler with base + at_activity
# Blend RMSE 0.610875 with tmp_agg I KNEW THEY WERE SHIT

In [3]:
file_name = 'base_feats'

train_ids = train_logs.id
test_ids = test_logs.id

logs = pd.concat([train_logs, test_logs], axis=0)
logs = normalise_up_down_times(logs)

preprocessor = Preprocessor(seed=42)
feats = preprocessor.make_feats(logs)
nan_cols = feats.columns[feats[feats['id'].isin(train_ids)].isna().any()].tolist()
feats = feats.drop(columns=nan_cols)

essays = getEssays(logs)
sent_feats = compute_sentence_aggregations(essays)
par_feats = compute_paragraph_aggregations(essays)
word_feats = create_word_length_features(essays, 'essay', 'id', 'essay_words')
file_name = 'base_feats'
feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

file_name = 'vectorised'
vector_feats = countvectorize_one_one(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

file_name = 'at_by_activity'
at_by_activity = process_action_time_activity(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

file_name = 'base_feats'
feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 33/33 [01:27<00:00,  2.66s/it, column=word_count_change100, method=kurt]         


Engineering activity counts data


100%|██████████| 2474/2474 [00:00<00:00, 12277.79it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering event counts data


...

100%|██████████| 2474/2474 [00:00<00:00, 13129.21it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering punctuation counts data


100%|██████████| 2474/2474 [00:00<00:00, 11571.04it/s]


Engineering input words data
Engineering ratios data


/root/Projects/Kaggle/linking-writing/m6_feats_comb.py:418: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
/root/Projects/Kaggle/linking-writing/m6_feats_comb.py:419: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
/root/Projects/Kaggle/linking-writing/m6_feats_comb.py:420: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

In [21]:
file_name = 'adj_eff_time'
feats = process_adjusted_eff_time(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

In [22]:
file_name = 'rep_cut'
feats = process_re_cut_essays(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

100%|██████████| 2470/2470 [00:00<00:00, 4315.15it/s]


,id,re_cut_count,re_cut_mean,re_cut_std,re_cut_max,re_cut_median,re_cut_sum,re_cut_last,re_cut_q1,re_cut_q3,re_cut_iqr,re_cut_min
0,001519c8,67.0,5.029851,3.279736,13.0,4.0,337.0,11.0,2.0,7.0,5.0,1.0
1,0022f953,37.0,5.864865,4.802558,23.0,4.0,217.0,2.0,3.0,8.0,5.0,1.0
2,0042269b,95.0,5.536842,3.924312,21.0,4.0,526.0,2.0,3.0,6.0,3.0,1.0
3,0059420b,37.0,3.027027,1.589663,7.0,3.0,112.0,4.0,2.0,4.0,2.0,1.0
4,0075873a,72.0,6.138889,3.132319,14.0,5.0,442.0,6.0,4.0,8.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2465,ffb8c745,144.0,5.520833,3.739730,18.0,4.0,795.0,14.0,3.0,8.0,5.0,1.0
2466,ffbef7e5,11.0,4.545455,2.659460,10.0,5.0,50.0,4.0,2.5,5.0,2.5,1.0
2467,ffccd6fd,14.0,4.571429,3.435433,12.0,3.5,64.0,5.0,2.0,5.0,3.0,1.0
2468,ffec5b38,45.0,6.577778,5.508212,24.0,4.0,296.0,1.0,3.0,8.0,5.0,1.0


In [ ]:
file_name = 'action_time_gap'
at_gap = process_feats_action_time_gap(logs)
at_gap.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

In [ ]:
file_name = 'action_time_gap_by_acti'
feats = process_feats_time_gap_activity(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

In [32]:
file_name = 'IKI'
IKI = logs.groupby(['id']).apply(calculate_pause_features).reset_index()
IKI.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

In [5]:
file_name = 'train_wc_chage'
feats = create_feats_wc_change(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

In [26]:
file_name = 'wpm_feats'
feats = wpm_feats(logs)
feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')

In [28]:
paste = logs[logs['activity'] == 'Paste'].copy()
paste['activity'] = 'Input'
paste_df = getEssays(paste)

100%|██████████| 317/317 [00:00<00:00, 19347.72it/s]


In [ ]:
0.609610
0.611009
0.616839